In [1]:
import pdfplumber
import pandas as pd
import re
from rapidfuzz import process, fuzz

In [ ]:
# deteccion de proveedor
def detect_provider(path):
    with pdfplumber.open(path) as pdf:
        first_page_text = pdf.pages[0].extract_text()
        
        if "TREOS" in first_page_text.upper():
            return "TALLER_A"
        elif "WAGEN" in first_page_text.upper():
            return "TALLER_B"
        elif "EQUIPO" in first_page_text.upper():
            return "TALLER_C"
        else:
            return "DESCONOCIDO"

## print(detect_provider("../data/presupuesto_A.pdf"))

TALLER_A


In [19]:
# extraccion caso taller A
def extract_taller_a(path):
    with pdfplumber.open(path) as pdf:
        table = pdf.pages[0].extract_table()
        df = pd.DataFrame(table[1:], columns=table[0]) 
        
        df.columns = [str(c).replace('\n', '') if c is not None else 'SIN_NOMBRE' for c in df.columns]
        df = df.map(lambda x: x.replace('\n', '') if isinstance(x, str) else x)
        
        df['PRECIO'] = df['PRECIO'].str.replace('.', '').str.replace(',', '.').astype(float)
        
        return df

df_treos = extract_taller_a("../data/presupuesto_A.pdf")
df_treos

,TIPO,DESCRIPCION,CANT.,DSCTO,PRECIO,SIN_NOMBRE,TOTAL
0,Servicio,CHAPA CARGO C.SEG,4.00,0 %,15454545.0,None,618181.80
1,Servicio,PINTURA CARGO C.SEG,8.00,0 %,15867769.0,None,1269421.52
2,Repuesto,GUARDABARRO DEL.DER.,1.00,0 %,36348701.0,None,363487.01
3,Repuesto,UNIT HEADLAMP W GAS,1.00,0 %,233355822.0,None,2333558.22
4,Repuesto,PUERTA DELANTERA,1.00,0 %,127736651.0,None,1277366.51
5,Repuesto,ESPEJO DERECHO,1.00,0 %,81953768.0,None,819537.68
6,Repuesto,MOLDURA MARCO VENTANA,1.00,0 %,5736545.0,None,57365.45
7,S U B T O T A L :,None,None,None,NaN,6738918.19 $,None
8,T O T A L S/IVA :,None,None,None,NaN,6738918.19 $,None
9,I V A (21%) :,None,None,None,NaN,1415172.82 $,None


In [ ]:
def transform_to_inspection(df, provider):
    output = []
    
    df_valid = df[
        df['DESCRIPCION'].notna() & 
        df['CANT.'].notna() & 
        ~df['TIPO'].str.contains("TOTAL|SUBTOTAL|IVA", case=False, na=False)
    ].copy()
    
    for _, row in df_valid.iterrows():
        try:
            cantidad = float(row['CANT.'])
        except (ValueError, TypeError):
            continue 
            
        item = {
            "descripcion": row['DESCRIPCION'],
            "tipo_item": "REPU" if "Repuesto" in str(row['TIPO']) else "MO",
            "precio_licitacion": 0.0 if "Repuesto" in str(row['TIPO']) else float(row.get('PRECIO', 0)),
            "cantidad": cantidad
        }
        
        desc_upper = str(row['DESCRIPCION']).upper()
        if "CHAPA" in desc_upper:
            item["horas_chapa"] = cantidad
        elif "PINTURA" in desc_upper:
            item["panos_pintura"] = cantidad
            
        output.append(item)
    
    return output


import pandas as pd

resultado = transform_to_inspection(df_treos, 'TALLER_A')
df_salida = pd.DataFrame(resultado)
print(df_salida.to_string(index=False))
print(df_salida.to_markdown(index=False))

          descripcion tipo_item  precio_licitacion  cantidad  horas_chapa  panos_pintura
    CHAPA CARGO C.SEG        MO         15454545.0       4.0          4.0            NaN
  PINTURA CARGO C.SEG        MO         15867769.0       8.0          NaN            8.0
 GUARDABARRO DEL.DER.      REPU                0.0       1.0          NaN            NaN
  UNIT HEADLAMP W GAS      REPU                0.0       1.0          NaN            NaN
     PUERTA DELANTERA      REPU                0.0       1.0          NaN            NaN
       ESPEJO DERECHO      REPU                0.0       1.0          NaN            NaN
MOLDURA MARCO VENTANA      REPU                0.0       1.0          NaN            NaN
| descripcion           | tipo_item   |   precio_licitacion |   cantidad |   horas_chapa |   panos_pintura |
|:----------------------|:------------|--------------------:|-----------:|--------------:|----------------:|
| CHAPA CARGO C.SEG     | MO          |         1.54545e+07 |         

In [ ]:
import pandas as pd

def transform_to_inspection(df, provider):
    output = []
    
    mask_valid = (
        df['DESCRIPCION'].notna() &
        df['CANT.'].notna() &
        ~df['TIPO'].astype(str).str.contains(r'TOTAL|SUBTOTAL|IVA|S U B T O T A L|T O T A L', case=False, na=False)
    )
    
    df_valid = df[mask_valid].copy()
    
    df_valid['CANT.']  = pd.to_numeric(df_valid['CANT.'],  errors='coerce')
    df_valid['PRECIO'] = pd.to_numeric(df_valid['PRECIO'], errors='coerce')
    
    df_valid = df_valid[df_valid['CANT.'].notna()]
    
    for _, row in df_valid.iterrows():
        cantidad = row['CANT.']
        precio_real = row['PRECIO'] if pd.notna(row['PRECIO']) else 0.0
        
        es_repuesto = "Repuesto" in str(row['TIPO'])
        
        item = {
            "descripcion": row['DESCRIPCION'].strip(),
            "tipo_item": "REPU" if es_repuesto else "MO",
            "precio_licitacion": 0.0 if es_repuesto else precio_real,
            "precio_repuesto": precio_real if es_repuesto else 0.0,
            "cantidad": float(cantidad),
        }
        
        desc_upper = str(row['DESCRIPCION']).upper()
        if "CHAPA" in desc_upper:
            item["horas_chapa"] = float(cantidad)
        elif "PINTURA" in desc_upper:
            item["panos_pintura"] = float(cantidad)
        
        output.append(item)
    
    return output


# Ejemplo de uso y visualización
resultado = transform_to_inspection(df_treos, 'TALLER_A')

df_salida = pd.DataFrame(resultado)

columns_order = ['tipo_item', 'cantidad', 'precio_licitacion', 'precio_repuesto', 'descripcion', 'horas_chapa', 'panos_pintura']
print(df_salida[columns_order].to_string(index=False, float_format="{:,.1f}".format))

print("\nVersión más legible (para consola o logs):")
print("-" * 100)
for item in resultado:
    precio_lic = f"{item['precio_licitacion']:>12,.0f}" if item['precio_licitacion'] > 0 else "          -"
    precio_rep = f"{item['precio_repuesto']:>12,.0f}" if item['precio_repuesto'] > 0 else "          -"
    
    print(f"{item['tipo_item']:<5} | cant: {item['cantidad']:>5.1f} | "
          f"licitación: {precio_lic} | repuesto: {precio_rep} | {item['descripcion']}")
    
    if "horas_chapa" in item:
        print(f"      └─ horas chapa: {item['horas_chapa']:.1f}")
    if "panos_pintura" in item:
        print(f"      └─ paños pintura: {item['panos_pintura']:.1f}")
print("-" * 100)

tipo_item  cantidad  precio_licitacion  precio_repuesto           descripcion  horas_chapa  panos_pintura
       MO       4.0       15,454,545.0              0.0     CHAPA CARGO C.SEG          4.0            NaN
       MO       8.0       15,867,769.0              0.0   PINTURA CARGO C.SEG          NaN            8.0
     REPU       1.0                0.0     36,348,701.0  GUARDABARRO DEL.DER.          NaN            NaN
     REPU       1.0                0.0    233,355,822.0   UNIT HEADLAMP W GAS          NaN            NaN
     REPU       1.0                0.0    127,736,651.0      PUERTA DELANTERA          NaN            NaN
     REPU       1.0                0.0     81,953,768.0        ESPEJO DERECHO          NaN            NaN
     REPU       1.0                0.0      5,736,545.0 MOLDURA MARCO VENTANA          NaN            NaN

Versión más legible (para consola o logs):
----------------------------------------------------------------------------------------------------
MO    |